In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

### 补充是否进入top 的信息

In [103]:
neuro = pd.read_csv('neuro_0203.csv')
neuro = neuro[(neuro.StopYear<=1995)&(neuro.StopYear>=1960)]

In [104]:
# interval = [[1961,1970],[1970,1980],[1980,1990],[1990,1995]]
interval = [[1975,1985],[1985,1990],[1990,1995]]
Top = {}
for itl in interval:
    i0 = itl[0];i1 = itl[1]
#     temp = np.array(neuro[['CID','YearlyCitationOfMentee']][(neuro.StopYear>=i0)&(neuro.StopYear<=i1)])
    temp = np.array(neuro[['CID','TotalCitationOfMentee']][(neuro.StopYear>=i0)&(neuro.StopYear<=i1)])
    CID = temp[:,0]
    YC = temp[:,1]
    mm = sorted(temp,key = lambda x:x[1],reverse=1)
    
    pt = 0.15   #进入 top 25
    th = mm[round(len(mm)*pt)][1]
    print(th)
    for i in range(len(CID)):
        if YC[i]>=th:
            Top[CID[i]]=1
        else:
            Top[CID[i]]=0

CID = neuro['CID']
b_or_s = []
for k in CID:
    b_or_s.append(Top[k])
neuro['Top25'] = b_or_s

41018
30178
21863


In [105]:
neuro_re = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','TotalCitationOfMentee','Top25','NumOfMenteesOfTheMenteeIn20']][(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             &(neuro.BigOrSmall!=-1)]
#neuro_era.BigOrSmall!=2)] #(neuro_era.TotalPubsOfMentor>0)&(neuro_era.CareerAgeOfMentorInTraining>-1)
                                    #(neuro_era.CareerAgeOfMentorInTraining<50)

In [106]:
neuro_re.shape

(3802, 23)

In [107]:
# neuro_re = neuro_re[(neuro_re.StopYear<=1995)&(neuro_re.StopYear>=1970)]

In [108]:
#测试大小团队中的生存率
all_big = neuro_re['MenteeID'][neuro_re.BigOrSmall==1]
all_small = neuro_re['MenteeID'][neuro_re.BigOrSmall==0]
sb = neuro_re['MenteeID'][neuro_re.Survival0==1]
sur_big = all_big[all_big.isin(sb)]
sur_small = all_small[all_small.isin(sb)]
print(len(sur_big)/len(all_big))
print(len(sur_small)/len(all_small))

0.4326241134751773
0.7033175355450237


In [109]:
neuro_re.to_csv('test0_0.25.csv',index=0)

##### 批量输出 cem 匹配截断点

In [110]:
cutpoint_list = ['First5YearPubsOfMentee','First5YearCitationOfMentee','First5YearCollaOfMentee',
                 'CollaPubsOfWithMentor',
                     'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor',
                     'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor',
                 'PubsOfMentorInTraning']

In [111]:
CutPoint = pd.DataFrame()
for k in cutpoint_list:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
    print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint[k] = cut    
    print(cut)

First5YearPubsOfMentee
[0, 3.0, 7.0, 109]
First5YearCitationOfMentee
[0, 149.0, 751.0, 65371]
First5YearCollaOfMentee
[0, 4.0, 10.0, 294]
CollaPubsOfWithMentor
[0, 0.0, 1.0, 68]
TotalPubsOfMentor
[0, 70.0, 175.0, 2791]
TotalCitationOfMentor
[0, 5781.0, 21872.0, 618069]
TotalCollaOfMentor
[0, 64.0, 187.0, 4303]
YearlyPubsOfMentor
[0.0, 1.6458333333333333, 3.58, 63.37209302325582]
YearlyCitationOfMentor
[0.0, 145.8913043478261, 456.44186046511635, 12773.363636363636]
YearllyCollaOfMentor
[0.0, 1.627906976744186, 4.040816326530612, 76.06976744186046]
PubsOfMentorInTraning
[0, 9.0, 22.0, 376]


In [112]:
#手动补充和修正0特多的变量的cutpoint
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
    print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
    print(cut)
    CutPoint[k] = cut
CutPoint['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
CutPoint['TotalCitationOfMentor'] =  [0, 1000, 5000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
CutPoint['YearllyPubsOfMentor'] =  [0, 4, 10, 64]
# CutPoint['PubsOfMentorInTraning'] =  [0, 6,19,300]

CollaPubsOfWithMentor
[0, 1, 3.0, 68]


In [113]:
# CutPoint['First5YearPubsOfMentee'] =  [0, 1, 5.0, 10.0, 435]

# CutPoint['First5YearCollaOfMentee'] =  [0, 8.0, 18.0, 60,139]
# CutPoint['First5YearCitationOfMentee'] =  [0, 100, 338.0, 1214.0, 101895]

In [114]:
CutPoint

,First5YearPubsOfMentee,First5YearCitationOfMentee,First5YearCollaOfMentee,CollaPubsOfWithMentor,TotalPubsOfMentor,TotalCitationOfMentor,TotalCollaOfMentor,YearlyPubsOfMentor,YearlyCitationOfMentor,YearllyCollaOfMentor,PubsOfMentorInTraning,CareerAgeOfMentorInTraining,YearllyPubsOfMentor
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0.0,0,0
1,3.0,149.0,4.0,1.0,70.0,1000,64.0,1.645833,145.891304,1.627907,9.0,15,4
2,7.0,751.0,10.0,3.0,175.0,5000,187.0,3.580000,456.441860,4.040816,22.0,25,10
3,109.0,65371.0,294.0,68.0,2791.0,309472,4303.0,63.372093,12773.363636,76.069767,376.0,50,64


In [115]:
CutPoint.to_csv('CutPoint0_0.25.csv',index=0)

In [116]:
# neuro_re1 = neuro_era[['NumOfMenteesOfTheMentor10','BirthYearOfMentee','First5YearPubsOfMentee','First5YearCitationOfMentee','First5YearCollaOfMentee',
#                      'BigOrSmall','TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor',
#                      'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','CareerAgeOfMentorInTraining',
#                     'Survival','NumOfMenteesOfTheMenteeIn20']][(neuro_era.CareerAgeOfMentorInTraining>-1)&(neuro_era.CareerAgeOfMentorInTraining<50)&
#                                 (neuro_era.TotalPubsOfMentor>0)&(neuro_era.BigOrSmall!=1)&(neuro_era.Survival==1)]

In [117]:
#限制条件比较少的情况
neuro_re1 = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','TotalCitationOfMentee','Top25',
                   'NumOfMenteesOfTheMenteeIn20']][#(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             (neuro.BigOrSmall!=-1)&(neuro.Survival0==1)]
    

In [146]:
#随机选取同等数量的小团队中的学生
neuro_re1_big = neuro_re1[neuro_re1.BigOrSmall==1]
neuro_re1_small = neuro_re1[neuro_re1.BigOrSmall==0]
print(len(neuro_re1_big),len(neuro_re1_small))
chosed_index = list(range(len(neuro_re1_small)))
random.shuffle(chosed_index)
temp = neuro_re1_small.iloc[chosed_index[:len(neuro_re1_big)],:]
neuro_re2 = pd.concat([neuro_re1_big, temp])

797 1804


In [147]:
neuro_re2

,MenteeID,MentorID,StopYear,BirthYearOfMentee,First5YearPubsOfMentee,First5YearCitationOfMentee,First5YearCollaOfMentee,CollaPubsOfWithMentor,BigOrSmall,TotalPubsOfMentor,...,YearlyPubsOfMentor,YearlyCitationOfMentor,YearllyCollaOfMentor,PubsOfMentorInTraning,Survival0,Survival1,YearlyCitationOfMentee,TotalCitationOfMentee,Top25,NumOfMenteesOfTheMenteeIn20
1,44,43,1985,1985.0,5,477,9,2,1,157,...,3.019231,461.019231,1.865385,30,1,1,1607.777778,57880,1,9
3,10,140,1989,1989.0,0,0,0,0,1,74,...,1.574468,275.021277,2.404255,4,1,1,0.000000,0,0,8
4,77,76,1995,1990.0,10,6032,11,7,1,922,...,17.396226,3740.547170,19.924528,140,1,1,722.781250,23129,1,9
6,92,43,1986,1982.0,14,3732,10,14,1,157,...,3.019231,461.019231,1.865385,31,1,1,376.820513,14696,0,2
7,100,102,1982,1982.0,36,15419,23,15,1,657,...,13.140000,2904.420000,7.420000,53,1,1,4397.050000,175882,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,32312,18981,1976,1976.0,4,1710,3,0,0,1,...,1.000000,8.000000,4.000000,0,1,0,534.046512,22964,0,0
4560,90659,90658,1979,1979.0,4,317,7,0,0,17,...,1.545455,9.090909,1.909091,0,1,1,527.935484,16366,0,1
5999,632978,233772,1989,1989.0,20,2315,29,0,0,20,...,0.909091,39.954545,1.227273,0,1,0,792.964286,22203,0,0
3374,26558,53391,1992,1991.0,11,2865,3,0,0,18,...,0.450000,36.075000,0.625000,6,1,1,332.935484,10321,0,6


In [118]:
CutPoint1 = pd.DataFrame()
for k in cutpoint_list:
    tp = np.array(neuro_re1[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
    print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint1[k] = cut    
    print(cut)

First5YearPubsOfMentee
[0, 4.0, 9.0, 63]
First5YearCitationOfMentee
[0, 276.0, 1002.0, 26513]
First5YearCollaOfMentee
[0, 6.0, 12.0, 139]
CollaPubsOfWithMentor
[0, 0.0, 1.0, 68]
TotalPubsOfMentor
[0, 63.0, 177.0, 2725]
TotalCitationOfMentor
[0, 4764.0, 21872.0, 618069]
TotalCollaOfMentor
[0, 60.0, 181.0, 3271]
YearlyPubsOfMentor
[0.0, 1.553846153846154, 3.583333333333333, 63.37209302325582]
YearlyCitationOfMentor
[0.0, 123.28571428571428, 450.50943396226415, 12503.260869565218]
YearllyCollaOfMentor
[0.0, 1.5227272727272727, 3.9777777777777783, 76.06976744186046]
PubsOfMentorInTraning
[0, 8.0, 22.0, 233]


In [119]:
#手动补充和修正0特多的变量的cutpoint
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
    print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
    print(cut)
    CutPoint1[k] = cut
CutPoint1['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
CutPoint1['TotalCitationOfMentor'] =  [0, 1000, 5000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
# CutPoint['YearllyCollaOfMentor'] =  [0, 2, 8, 88]
# CutPoint['PubsOfMentorInTraning'] =  [0, 6,19,300]
CutPoint['YearllyPubsOfMentor'] =  [0, 4, 10, 64]

CollaPubsOfWithMentor
[0, 1, 3.0, 68]


In [120]:
neuro_re1.shape

(2601, 23)

In [148]:
# neuro_re1.to_csv('test1_0.25.csv',index=0)
neuro_re2.to_csv('test1_0.25.csv',index=0)

In [149]:
CutPoint1.to_csv('CutPoint1_0.25.csv',index=0)